In [ ]:
!pip install transformers  gradio diffusers torch gtts Pillow deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB

In [ ]:
# Import necessary libraries
from transformers import BlipProcessor, BlipForConditionalGeneration  # For image captioning
from PIL import Image  # To handle image processing
import gradio as gr  # Gradio for creating web interface
from diffusers import StableDiffusionPipeline  # For generating images from text
import torch  # To enable GPU acceleration
from deep_translator import GoogleTranslator  # For automatic translation
from gtts import gTTS  # Google Text-to-Speech for generating audio from text
import os  # For file handling

# Function to translate text to a target language
def translate_text(text, target_language):
    return GoogleTranslator(source='auto', target=target_language).translate(text)

# Load the BLIP model and processor for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load the Stable Diffusion pipeline for text-to-image generation
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16
)

# Check if GPU is available and move models to GPU for faster computation
if torch.cuda.is_available():
    model.to("cuda")
    pipe.to("cuda")

# Function for generating captions from images, with text-to-speech
def generate_caption(image, target_language):
    inputs = processor(image, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    out = model.generate(**inputs)

    caption = processor.decode(out[0], skip_special_tokens=True)

    translated_caption = translate_text(caption, target_language)

    tts = gTTS(text=translated_caption, lang=target_language)
    tts.save("output.mp3")

    return translated_caption, "output.mp3"

# Function for generating images from text descriptions
def generate_image_from_text(description, image_input=None):
    translated_description = translate_text(description, "en")

    if image_input is not None:
        inputs = processor(image_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
        out = model.generate(**inputs)
        image_caption = processor.decode(out[0], skip_special_tokens=True)
        translated_description += " " + image_caption  # Append the image caption to the text

    image = pipe(translated_description, num_inference_steps=70, guidance_scale=6.5).images[0]

    return image

# Gradio interface for image captioning with Text-to-Speech
iface_image_to_text = gr.Interface(
    fn=generate_caption,  # The function to execute
    inputs=[
        gr.Image(type="pil"),  # Image input
        gr.Dropdown(
            choices=[("English", "en"), ("Arabic", "ar"), ("French", "fr"), ("Spanish", "es"), ("German", "de")],
            label="Select Output Language",
            value="ar"  # Default language is set to Arabic
        )
    ],
    outputs=[
        "text",  # Text output (caption)
        gr.Audio(type="filepath")  # Audio output (speech of caption)
    ],
    title="Image Captioning with Speech Output",
    description="Upload an image and get a caption in the selected language with audio output.",

    # Adding examples (local images paths or URLs)
    examples=[
        ["https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQoQVWVsG_u0dE0IDFQTszJRcSz1kl3PlXb_g&s", "ar"],
        ["https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSwf8ck2f453p_Nt3jJjo6Xfl5Iu4IpprnLJw&s", "en"]
    ]
)


# Example inputs for text-to-image generation
examples = [
    ["A beautiful sunset over the mountains."],
    ["A futuristic cityscape with flying cars."],
    ["منظر للبحر مع قارب صغير يطفو على سطح الماء"],
    ["رجل يقرأ كتابًا تحت شجرة في يوم مشمس"],
    ["A spaceship landing on Mars."],
    ["سفينة تبحر في محيط هادئ تحت سماء زرقاء صافية"],
]

# Gradio interface for text-to-image generation with examples
iface_text_to_image = gr.Interface(
    fn=generate_image_from_text,
    inputs=[
        gr.Textbox(label="Enter your description", placeholder="Type your description here..."),
        gr.Image(type="pil", label="Optional: Upload an image")
    ],
    outputs="image",
    title="Text-to-Image Generation",
    description="Enter a description or upload an image to generate an image.",
    examples=examples  # Added examples for quick access
)

# Combining both interfaces into a tabbed Gradio interface
iface_combined = gr.TabbedInterface(
    [iface_image_to_text, iface_text_to_image],
    tab_names=["Image Captioning with Speech", "Text-to-Image"]
)

# Launch the Gradio interface
iface_combined.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b317399c3162fba189.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
